In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# === 1. Install dependensi yang diperlukan ===
!pip install -q langchain langchain-community langchain-huggingface chromadb sentence-transformers unstructured beautifulsoup4 google-generativeai


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 5.4 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 50.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 69.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.7/19.7 MB 79.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 24.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 284.2/284.2 kB 20.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 73.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.1/103.1 kB 8.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.5/16.5 MB 38.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.6/65.6 kB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
data = "/content/drive/MyDrive/RisetQA/dataset_qa.json"

In [ ]:
from langchain.schema import Document
import json

# Misal dataset kamu disimpan di file: bpjs_qa.json
with open(data, "r", encoding="utf-8") as f:
    data = json.load(f)

# Konversi ke list Document
documents = []
for item in data:
    doc = Document(
        page_content=f"Pertanyaan: {item['question']}\nJawaban: {item['answer']}",
        metadata={"source": "bpjs_qa_dataset"}
    )
    documents.append(doc)

In [ ]:
documents

[Document(metadata={'source': 'bpjs_qa_dataset'}, page_content='Pertanyaan: Bagaimana kebijakan pengenaan  denda  akibat keterlambatan pembayaran iuran Jaminan Kesehatan   ?\nJawaban: 1 . Bahwa apabila dalam waktu 45 (empat puluh lima) hari sejak status kepesertaan aktif kembali, Peserta wajib membayar denda kepada BPJS Kesehatan untuk satu kali rawat inap tingkat lanjutan yang diperolehnya.\n2. Denda tersebut dikenakan atas pelayanan kesehatan yang tagihan dendanya pertama kali dicetak.\n3. Denda sebesar 5% (lima persen) dari perkiraan biaya paket Indonestan Case Based  Groups berdasarkan diagnosa dan prosedur awal untuk setiap bulan tertunggak dengan ketentuan:\n    a. Jumlah bulan tertunggak paling banyak 12 (dua belas) bulan; dan\n    b. Besar denda paling tinggi Rp20.000.000,00 (dua puluh juta rupiah).'),
 Document(metadata={'source': 'bpjs_qa_dataset'}, page_content='Pertanyaan: Dokumen Pendukung yang Dibutuhkan Dalam Mengajukan Penjaminan 6 Bulan Bagi Pekerja yang terkena PHK\nJ

In [ ]:
# Tidak perlu split jika tiap QA sudah singkat
chunks = documents

In [ ]:
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_community.vectorstores import Chroma

embedding_model = HuggingFaceEmbeddings(
    model_name="sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2"
)

vectorstore = Chroma.from_documents(
    documents,
    embedding_model,
    persist_directory="./chroma_bpjs"
)

retriever = vectorstore.as_retriever()

In [ ]:
# Convert chunks ke embedding
embeddings = embedding_model.embed_documents([chunk.page_content for chunk in chunks])

print(f"Jumlah embedding: {len(embeddings)}, dimensi: {len(embeddings[0])}")

Jumlah embedding: 132, dimensi: 384


In [ ]:
!pip install -q google-generativeai langchain_google_genai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.0/42.0 kB 3.1 MB/s eta 0:00:00


In [ ]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_google_genai import ChatGoogleGenerativeAI

prompt = ChatPromptTemplate.from_template(
    """Anda adalah asisten BPJS Kesehatan. Gunakan konteks untuk menjawab pertanyaan.
Jika tidak tahu, katakan "Saya tidak tahu". Jawab dengan maksimal 4 kalimat.

Pertanyaan: {question}
Konteks: {context}

Jawaban:"""
)

llm = ChatGoogleGenerativeAI(model="gemini-1.5-flash", google_api_key="***********")
chain = prompt | llm

In [ ]:
query = "Siapakah yang dimaksud dengan Peserta?"
docs = retriever.invoke(query)
response = chain.invoke({"question": query, "context": docs})
print(response.content)

Peserta adalah individu yang terdaftar dalam program BPJS Kesehatan.  Mereka berhak mendapatkan layanan kesehatan sesuai dengan kelas kepesertaan yang dipilih.  Dalam konteks PPU, peserta juga mencakup anggota keluarga. Pemberi kerja menanggung denda layanan bagi peserta PPU.


In [ ]:
query = "Apa yang dimaksud dengan Virtual Account (VA)"
docs = retriever.invoke(query)
response = chain.invoke({"question": query, "context": docs})
print(response.content)

Virtual Account (VA) adalah nomor rekening virtual yang disediakan BPJS Kesehatan.  Nomor ini digunakan sebagai tujuan pembayaran iuran kesehatan bagi perorangan maupun entitas.  VA memudahkan proses pembayaran iuran.  Pembayaran dapat dilakukan melalui berbagai metode.


In [ ]:
!pip install rouge_score

  Preparing metadata (setup.py) ... done
  Created wheel for rouge_score: filename=rouge_score-0.1.2-py3-none-any.whl size=24934 sha256=04acf067377509ec97f97f8c38262d40fa153cffc6da645fdf79455e95e6a7e6
  Stored in directory: /root/.cache/pip/wheels/1e/19/43/8a442dc83660ca25e163e1bd1f89919284ab0d0c1475475148
Successfully built rouge_score


In [ ]:
from rouge_score import rouge_scorer

scorer = rouge_scorer.RougeScorer(['rouge1', 'rougeL'], use_stemmer=True)

reference = "Peserta adalah Semua  penduduk Indonesia, termasuk orang asing yang telah bekerja paling singkat 6 (enam) bulan di Indonesia dan telah membayar iuran"
generated = response.content
scores = scorer.score(reference, generated)
print(scores)
# Output: {'rouge1': f1=0.5, 'rougeL': f1=0.6}

{'rouge1': Score(precision=0.08823529411764706, recall=0.13636363636363635, fmeasure=0.10714285714285714), 'rougeL': Score(precision=0.08823529411764706, recall=0.13636363636363635, fmeasure=0.10714285714285714)}


In [ ]:
!pip install bert_score

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.1/61.1 kB 4.4 MB/s eta 0:00:00


In [ ]:
from bert_score import score

references = ["Apa yang dimaksud dengan Virtual Account (VA)"]
generated = [response.content]

P, R, F1 = score(generated, references, lang="id")  # gunakan 'en' jika bahasa Inggris
print(f"BERTScore F1: {F1.mean().item():.3f}")  # Misal: 0.92 → sangat mirip secara makna

BERTScore F1: 0.615


In [ ]:
import pandas as pd
data = pd.read_json("/content/drive/MyDrive/RisetQA/dataset_qa.json")
data

,question,answer
0,Bagaimana kebijakan pengenaan denda akibat k...,1 . Bahwa apabila dalam waktu 45 (empat puluh ...
1,Dokumen Pendukung yang Dibutuhkan Dalam Mengaj...,1. Dokumen data Peserta dan anggota keluar...
2,Bagaimana Ketentuan Mengenai Penjaminan 6 Bula...,a.\tPeserta PPU yang mengalami PHK dan membutu...
3,Mekanisme Pelaporan Pekerja Terkena PHK,"a.\tBukti PHK sesuai ketentuan, disampaikan ol..."
4,Bukti Pekerja terkena PHK yang mendapat Jamina...,1. dibuktikan dengan:\na.\tbukti diterimanya P...
...,...,...
127,Siapa yang dimaksud Pekerja Bukan Penerima Upah,Pekerja Bukan Penerima Upah adalah setiap oran...
128,Siapa yang dimaksud dengan Pekerja,Pekerja adalah setiap orang yang bekerja denga...
129,Siapakah yang dimaksud dengan Peserta,"Peserta adalah Semua penduduk Indonesia, term..."
130,Apakah yang dimaksud dengan Iuran Jaminan Kese...,Iuran Jaminan Kesehatan adalah sejumlah uang y...


In [ ]:
from bert_score import score
import numpy as np

# Inisialisasi list untuk menyimpan skor
f1_scores = []

for q, a in zip(data['question'], data['answer']):
    query = q
    docs = retriever.invoke(query)
    response = chain.invoke({"question": query, "context": docs})

    references = [a]  # Jawaban referensi
    generated = [response.content]  # Jawaban yang dihasilkan

    # Hitung BERTScore
    P, R, F1 = score(generated, references, lang="id")  # gunakan 'en' jika bahasa Inggris
    current_f1 = F1.mean().item()
    f1_scores.append(current_f1)

    # Cetak skor untuk pertanyaan saat ini
    print(f"BERTScore F1 for question '{q}': {current_f1:.3f}")

# Hitung dan cetak rata-rata setelah loop selesai
if f1_scores:  # Pastikan list tidak kosong
    average_f1 = np.mean(f1_scores)
    print(f"\nAverage BERTScore F1 across all questions: {average_f1:.3f}")
else:
    print("No scores were calculated.")

BERTScore F1 for question 'Bagaimana kebijakan pengenaan  denda  akibat keterlambatan pembayaran iuran Jaminan Kesehatan   ?': 0.762
BERTScore F1 for question 'Dokumen Pendukung yang Dibutuhkan Dalam Mengajukan Penjaminan 6 Bulan Bagi Pekerja yang terkena PHK': 0.759
BERTScore F1 for question 'Bagaimana Ketentuan Mengenai Penjaminan 6 Bulan Bagi Pekerja yang terkena PHK': 0.741
BERTScore F1 for question 'Mekanisme Pelaporan Pekerja Terkena PHK': 0.816
BERTScore F1 for question 'Bukti Pekerja terkena PHK yang mendapat Jaminan 6 Bulan tanpa membayar iuran?': 0.770
BERTScore F1 for question 'cara Melakukan Perubahan Fasilitas Kesehatan Tingkat Pertama (FKTP)': 0.640
BERTScore F1 for question 'Bagaimana menentukan Fasiltas Kesehatan Tingkat Pertama (FKTP)  bagi  Peserta pertama kali ?': 0.812
BERTScore F1 for question 'Bagaimana keikutsertaan Penduduk Indonesia dalam Program JKN ?': 0.762
BERTScore F1 for question 'Siapa yang menanggung denda layanan bagi peserta PPU ?': 0.722
BERTScore F1

  quota_metric: "generativelanguage.googleapis.com/generate_content_free_tier_requests"
  quota_id: "GenerateRequestsPerMinutePerProjectPerModel-FreeTier"
  quota_dimensions {
    key: "model"
    value: "gemini-1.5-flash"
  }
  quota_dimensions {
    key: "location"
    value: "global"
  }
  quota_value: 15
}
, links {
  description: "Learn more about Gemini API quotas"
  url: "https://ai.google.dev/gemini-api/docs/rate-limits"
}
, retry_delay {
  seconds: 7
}
].


BERTScore F1 for question 'Apa syarat pendaftaran Rumah Ibadah Asosiasi / organisasi': 0.753


ResourceExhausted: 429 You exceeded your current quota, please check your plan and billing details. For more information on this error, head to: https://ai.google.dev/gemini-api/docs/rate-limits. [violations {
  quota_metric: "generativelanguage.googleapis.com/generate_content_free_tier_requests"
  quota_id: "GenerateRequestsPerMinutePerProjectPerModel-FreeTier"
  quota_dimensions {
    key: "model"
    value: "gemini-1.5-flash"
  }
  quota_dimensions {
    key: "location"
    value: "global"
  }
  quota_value: 15
}
, links {
  description: "Learn more about Gemini API quotas"
  url: "https://ai.google.dev/gemini-api/docs/rate-limits"
}
, retry_delay {
  seconds: 5
}
]

In [ ]:
from bert_score import score

# Inisialisasi list untuk menyimpan skor
f1_scores = []
num_samples = 10  # Jumlah sampel yang ingin dihitung

for i, (q, a) in enumerate(zip(data['question'], data['answer'])):
    if i >= num_samples:
        break  # Berhenti setelah 10 iterasi

    query = q
    docs = retriever.invoke(query)
    response = chain.invoke({"question": query, "context": docs})

    references = [a]
    generated = [response.content]

    P, R, F1 = score(generated, references, lang="id")
    current_f1 = F1.mean().item()
    f1_scores.append(current_f1)

    print(f"BERTScore F1 untuk pertanyaan ke-{i+1} '{q}': {current_f1:.3f}")

# Hitung rata-rata dari 10 sampel
if f1_scores:
    average_f1 = sum(f1_scores) / len(f1_scores)
    print(f"\nRata-rata BERTScore F1 dari {len(f1_scores)} pertanyaan pertama: {average_f1:.3f}")
else:
    print("Tidak ada skor yang dihitung.")

  quota_metric: "generativelanguage.googleapis.com/generate_content_free_tier_requests"
  quota_id: "GenerateRequestsPerDayPerProjectPerModel-FreeTier"
  quota_dimensions {
    key: "model"
    value: "gemini-1.5-flash"
  }
  quota_dimensions {
    key: "location"
    value: "global"
  }
  quota_value: 50
}
, links {
  description: "Learn more about Gemini API quotas"
  url: "https://ai.google.dev/gemini-api/docs/rate-limits"
}
, retry_delay {
  seconds: 43
}
].


ResourceExhausted: 429 You exceeded your current quota, please check your plan and billing details. For more information on this error, head to: https://ai.google.dev/gemini-api/docs/rate-limits. [violations {
  quota_metric: "generativelanguage.googleapis.com/generate_content_free_tier_requests"
  quota_id: "GenerateRequestsPerDayPerProjectPerModel-FreeTier"
  quota_dimensions {
    key: "model"
    value: "gemini-1.5-flash"
  }
  quota_dimensions {
    key: "location"
    value: "global"
  }
  quota_value: 50
}
, links {
  description: "Learn more about Gemini API quotas"
  url: "https://ai.google.dev/gemini-api/docs/rate-limits"
}
, retry_delay {
  seconds: 41
}
]